In [1]:
import os
# os.chdir('./net2neuro')
from tqdm import tqdm
import pickle
import mne
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
# from simpleconv import SimpleConv
import torch.optim as optim
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
base_dir = '/scratch/jd5697/cv_project'

In [4]:
embeddings = np.load(os.path.join(base_dir, 'image_embeddings_vit.npy'), allow_pickle=True).item()

In [5]:
n_participants = 4

In [6]:
def load_epochs(preproc_dir,all_epochs = []):
    for p in range(1,n_participants+1):
        epochs1 = mne.read_epochs(f'{preproc_dir}/preprocessed_P{str(p)}-epo.fif', preload=False)
        epochs2 = mne.read_epochs(f'{preproc_dir}/preprocessed_P{str(p)}-epo-1.fif', preload=False)
        epochs3 = mne.read_epochs(f'{preproc_dir}/preprocessed_P{str(p)}-epo-2.fif', preload=False)
        epochs = mne.concatenate_epochs([epochs1, epochs2, epochs3])
        all_epochs.append(epochs)
        del epochs1, epochs2, epochs3, epochs
    return all_epochs

In [7]:
base_path = "/scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new"
# all_epochs = mne.read_epochs(f'{base_path}/preprocessed_P1-epo.fif'\
#                              , preload = False)

all_epochs = load_epochs(base_path)

Reading /scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new/preprocessed_P1-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
Adding metadata with 18 columns
9016 matching events found
No baseline correction applied
0 projection items activated
Reading /scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new/preprocessed_P1-epo-1.fif ...


/state/partition1/job-54765026/ipykernel_3852051/1705987501.py:4: RuntimeWarning: This filename (/scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new/preprocessed_P1-epo-1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs2 = mne.read_epochs(f'{preproc_dir}/preprocessed_P{str(p)}-epo-1.fif', preload=False)


    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
Adding metadata with 18 columns
9016 matching events found
No baseline correction applied
0 projection items activated
Reading /scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new/preprocessed_P1-epo-2.fif ...


/state/partition1/job-54765026/ipykernel_3852051/1705987501.py:5: RuntimeWarning: This filename (/scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new/preprocessed_P1-epo-2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs3 = mne.read_epochs(f'{preproc_dir}/preprocessed_P{str(p)}-epo-2.fif', preload=False)


    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
Adding metadata with 18 columns
9016 matching events found
No baseline correction applied
0 projection items activated
Loading data for 9016 events and 181 original time points ...
Loading data for 9016 events and 181 original time points ...
Loading data for 9016 events and 181 original time points ...
Adding metadata with 18 columns
27048 matching events found
No baseline correction applied
Reading /scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new/preprocessed_P2-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
Adding metadata with 18 columns
9016 matching events found
No baseline correction applied
0 projection items activated
Reading /scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new/preprocessed_P2-epo-1.fif ...


...

Adding metadata with 18 columns
9016 matching events found
No baseline correction applied
0 projection items activated
Loading data for 9016 events and 181 original time points ...
Loading data for 9016 events and 181 original time points ...
Loading data for 9016 events and 181 original time points ...
Adding metadata with 18 columns
27048 matching events found
No baseline correction applied
Reading /scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new/preprocessed_P4-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
Adding metadata with 18 columns
9016 matching events found
No baseline correction applied
0 projection items activated
Reading /scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new/preprocessed_P4-epo-1.fif ...


/state/partition1/job-54765026/ipykernel_3852051/1705987501.py:4: RuntimeWarning: This filename (/scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new/preprocessed_P4-epo-1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs2 = mne.read_epochs(f'{preproc_dir}/preprocessed_P{str(p)}-epo-1.fif', preload=False)


    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
Adding metadata with 18 columns
9016 matching events found
No baseline correction applied
0 projection items activated
Reading /scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new/preprocessed_P4-epo-2.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available


/state/partition1/job-54765026/ipykernel_3852051/1705987501.py:5: RuntimeWarning: This filename (/scratch/jd5697/cv_project/THINGSmeg/ds004212/derivatives/preprocessed/new/preprocessed_P4-epo-2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs3 = mne.read_epochs(f'{preproc_dir}/preprocessed_P{str(p)}-epo-2.fif', preload=False)


Adding metadata with 18 columns
9016 matching events found
No baseline correction applied
0 projection items activated
Loading data for 9016 events and 181 original time points ...
Loading data for 9016 events and 181 original time points ...
Loading data for 9016 events and 181 original time points ...
Adding metadata with 18 columns
27048 matching events found
No baseline correction applied


In [8]:
for epoch in all_epochs:
    epoch.metadata['image_path'] = epoch.metadata.apply(
        lambda row: row['image_path'][11:] if row['trial_type'] != 'test' else row['image_path'][16:], axis=1
    )
    # To make image path of test compatible 
    epoch.metadata['image_path'] = epoch.metadata.apply(
        lambda row: f"{'_'.join(row['image_path'].split('_')[:-1])}/{row['image_path']}" if row['trial_type'] == 'test' else row['image_path'], axis=1
    )
    
    
    #f"{x.split('_')[0]}/{x}"

In [9]:
all_epochs[0].metadata['subject_index'] = 0
all_epochs[1].metadata['subject_index'] = 1
all_epochs[2].metadata['subject_index'] = 2
all_epochs[3].metadata['subject_index'] = 3

In [10]:
all_epochs[1].info['dev_head_t'] =  all_epochs[0].info['dev_head_t']
all_epochs[2].info['dev_head_t'] =  all_epochs[0].info['dev_head_t']
all_epochs[3].info['dev_head_t'] =  all_epochs[0].info['dev_head_t']

In [11]:
print(len(all_epochs[0].metadata['trial_type'].unique()))

# Get the number of values for each trial type in the metadata
for i in range(n_participants):
    print(all_epochs[i].metadata['trial_type'].value_counts())

3
trial_type
exp      22248
catch     2400
test      2400
Name: count, dtype: int64
trial_type
exp      22248
catch     2400
test      2400
Name: count, dtype: int64
trial_type
exp      22248
test      2400
catch     2400
Name: count, dtype: int64
trial_type
exp      22248
catch     2400
test      2400
Name: count, dtype: int64


In [12]:
# Drop all the epochs where the trial type is catch
for i in range(n_participants):
    all_epochs[i] = all_epochs[i][all_epochs[i].metadata['trial_type'] != 'catch']
    
# Reset the metadata index
for i in range(n_participants):
    all_epochs[i].metadata = all_epochs[i].metadata.reset_index(drop=True)
    
# Get the number of values for each trial type in the metadata
for i in range(n_participants):
    print(all_epochs[i].metadata['trial_type'].value_counts())

Replacing existing metadata with 19 columns
Replacing existing metadata with 19 columns
Replacing existing metadata with 19 columns
Replacing existing metadata with 19 columns
trial_type
exp     22248
test     2400
Name: count, dtype: int64
trial_type
exp     22248
test     2400
Name: count, dtype: int64
trial_type
exp     22248
test     2400
Name: count, dtype: int64
trial_type
exp     22248
test     2400
Name: count, dtype: int64


In [13]:
def partition_data(epoch):
    
        
    # Create separate epochs for training and testing
    epoch_train = epoch[epoch.metadata['trial_type'] != 'test']
    epoch_test = epoch[epoch.metadata['trial_type'] == 'test']
    
    # Reset the metadata index
    epoch_train.metadata = epoch_train.metadata.reset_index(drop=True)
    
    # Add a column to metadata indicating if image_path is a duplicate
    epoch_test.metadata['is_duplicate'] = epoch_test.metadata.duplicated(subset='image_path', keep='first')
    epoch_test = epoch_test[epoch_test.metadata['is_duplicate'] == False]
    
    epoch_test.metadata = epoch_test.metadata.reset_index(drop=True)
    
    return epoch_train, epoch_test
    

In [14]:
epochs_train = []
epochs_test = []

In [15]:
for i in range(n_participants):
    epoch_train, epoch_test = partition_data(all_epochs[i])
    epochs_train.append(epoch_train)
    epochs_test.append(epoch_test)
    

Replacing existing metadata with 19 columns
Replacing existing metadata with 20 columns
Replacing existing metadata with 19 columns
Replacing existing metadata with 20 columns
Replacing existing metadata with 19 columns
Replacing existing metadata with 20 columns
Replacing existing metadata with 19 columns
Replacing existing metadata with 20 columns


In [16]:
for epoch in epochs_train:
    print(len(epoch))
    
for epoch in epochs_test:
    print(len(epoch))

22248
22248
22248
22248
200
200
200
200


In [17]:
del all_epochs

In [18]:
del epochs_train

In [19]:
#epochs_train = mne.concatenate_epochs(epochs_train)
epochs_test = mne.concatenate_epochs(epochs_test)

Adding metadata with 20 columns
800 matching events found
No baseline correction applied


In [20]:
print(epochs_test.metadata['image_path'].value_counts())

image_path
candelabra/candelabra_14s.jpg                4
baby/baby_17s.jpg                            4
broccoli/broccoli_14s.jpg                    4
seesaw/seesaw_13s.jpg                        4
aircraft_carrier/aircraft_carrier_22s.jpg    4
                                            ..
limousine/limousine_15s.jpg                  4
bell/bell_14n.jpg                            4
floss/floss_13s.jpg                          4
typewriter/typewriter_13s.jpg                4
starfish/starfish_15n.jpg                    4
Name: count, Length: 200, dtype: int64


In [21]:
embeddings_train = {}
embeddings_test = {}

In [21]:
for image_path in epochs_train.metadata['image_path']:
    embeddings_train[image_path] = embeddings[image_path]


In [22]:
for image_path in epochs_test.metadata['image_path']:
    embeddings_test[image_path] = embeddings[image_path]

In [23]:
print(len(embeddings_train))
print(len(embeddings_test))

0
200


In [22]:
with open('valid_epochs_small_train_resplit.pickle', 'wb') as f:
    pickle.dump(epochs_train, f, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
#check shape
with open('valid_epochs_small_train_resplit.pickle', 'rb') as f:
    epochs_train = pickle.load(f)

print(epochs_train.get_data().shape)

(88992, 272, 181)


In [23]:
del epochs_train

In [24]:
with open('valid_epochs_small_test_resplit.pickle', 'wb') as f:
    pickle.dump(epochs_test, f, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
#check shape
with open('valid_epochs_small_test_resplit.pickle', 'rb') as f:
    epochs_test = pickle.load(f)

print(epochs_test.get_data().shape)

(800, 272, 181)


In [25]:
pwd

'/scratch/dm5927/cv_project/net2neuro'

In [26]:
np.save('./image_embeddings_vit_small_train_resplit.npy', embeddings_train)

In [24]:
np.save(os.path.join(base_dir, 'image_embeddings_vit_test_small_resplit.npy'), embeddings_test)